In [22]:
from sympy import symbols, Eq, linear_eq_to_matrix, Matrix

def montar_sistema(equacoes):
    n = len(equacoes)
    variaveis = symbols(f'x1:{n+1}')
    eqs = []
    for eq in equacoes:
        lhs, rhs = eq.split('=')
        eqs.append(Eq(eval(lhs, dict(zip([str(v) for v in variaveis], variaveis))), eval(rhs)))
    A, b = linear_eq_to_matrix(eqs, variaveis)
    return A, b, variaveis

def imprimir_matriz_b(A, b):
    n = len(A)
    print("Matriz A | Vetor b")
    for i in range(n):
        linha = ' '.join(f"{A[i][j]:10.4f}" for j in range(n))
        print(f"[ {linha} ] | {b[i]:10.4f}")
    print()

def eliminacao_gauss_com_passos(A, b):
    n = A.shape[0]
    A = [[float(A[i, j]) for j in range(n)] for i in range(n)]
    b = [float(b[i, 0]) for i in range(n)]

    print("Sistema inicial:")
    imprimir_matriz_b(A, b)

    # Eliminação para triangularização
    for k in range(n):
        if abs(A[k][k]) < 1e-12:
            raise ZeroDivisionError("Pivô zero detectado, pivotação necessária.")
        for i in range(k + 1, n):
            fator = A[i][k] / A[k][k]
            print(f"Multiplicando linha {k} por {fator:.4f} e subtraindo da linha {i}")
            for j in range(k, n):
                A[i][j] -= fator * A[k][j]
            b[i] -= fator * b[k]
            imprimir_matriz_b(A, b)

    # Retro-substituição
    x = [0] * n
    for i in range(n - 1, -1, -1):
        soma = sum(A[i][j] * x[j] for j in range(i + 1, n))
        x[i] = (b[i] - soma) / A[i][i]

    return x, A, b

def calcular_residuo(A, x, b):
    n = len(A)
    residuo = [0] * n
    for i in range(n):
        soma = 0
        for j in range(n):
            soma += A[i][j] * x[j]
        val = soma - b[i]
        residuo[i] = 0 if abs(val) < 1e-5 else val
    return residuo

# Sistema
equacoes = [
    '1*x1 - 3*x2 + 2*x3 = 11',
    '-2*x1 + 8*x2 - 3*x3 = -15',
    '4*x1 - 6*x2 + 5*x3 = 29'
]

A, b, vars = montar_sistema(equacoes)
solucao, A_triangular, b_modificado = eliminacao_gauss_com_passos(A, b)

print("Solução do sistema:")
for var, val in zip(vars, solucao):
    print(f"{var} = {val:.4f}")

residuo = calcular_residuo(A_triangular, solucao, b_modificado)
print("\nResíduo (Ax - b):")
for i, r in enumerate(residuo):
    print(f"r{i+1} = {r:.4e}" if r != 0 else f"r{i+1} = 0")


Sistema inicial:
Matriz A | Vetor b
[     1.0000    -3.0000     2.0000 ] |    11.0000
[    -2.0000     8.0000    -3.0000 ] |   -15.0000
[     4.0000    -6.0000     5.0000 ] |    29.0000

Multiplicando linha 0 por -2.0000 e subtraindo da linha 1
Matriz A | Vetor b
[     1.0000    -3.0000     2.0000 ] |    11.0000
[     0.0000     2.0000     1.0000 ] |     7.0000
[     4.0000    -6.0000     5.0000 ] |    29.0000

Multiplicando linha 0 por 4.0000 e subtraindo da linha 2
Matriz A | Vetor b
[     1.0000    -3.0000     2.0000 ] |    11.0000
[     0.0000     2.0000     1.0000 ] |     7.0000
[     0.0000     6.0000    -3.0000 ] |   -15.0000

Multiplicando linha 1 por 3.0000 e subtraindo da linha 2
Matriz A | Vetor b
[     1.0000    -3.0000     2.0000 ] |    11.0000
[     0.0000     2.0000     1.0000 ] |     7.0000
[     0.0000     0.0000    -6.0000 ] |   -36.0000

Solução do sistema:
x1 = 0.5000
x2 = 0.5000
x3 = 6.0000

Resíduo (Ax - b):
r1 = 0
r2 = 0
r3 = 0


In [21]:
from sympy import symbols, Eq, linear_eq_to_matrix

def montar_sistema(equacoes):
    n = len(equacoes)
    variaveis = symbols(f'x1:{n+1}')
    eqs = []
    for eq in equacoes:
        lhs, rhs = eq.split('=')
        eqs.append(Eq(eval(lhs, dict(zip([str(v) for v in variaveis], variaveis))), eval(rhs)))
    A, b = linear_eq_to_matrix(eqs, variaveis)
    return A, b, variaveis

def imprimir_matriz_b(A, b):
    n = len(A)
    print("Matriz A | Vetor b")
    for i in range(n):
        linha = ' '.join(f"{A[i][j]:10.4f}" for j in range(n))
        print(f"[ {linha} ] | {b[i]:10.4f}")
    print()

def eliminacao_gauss_pivoteamento(A, b):
    n = A.shape[0]
    A = [[float(A[i, j]) for j in range(n)] for i in range(n)]
    b = [float(b[i, 0]) for i in range(n)]

    print("Sistema inicial:")
    imprimir_matriz_b(A, b)

    for k in range(n):
        # Pivoteamento parcial
        pivo_linha = max(range(k, n), key=lambda i: abs(A[i][k]))
        if abs(A[pivo_linha][k]) < 1e-12:
            raise ZeroDivisionError("Pivô zero detectado, sistema singular ou pivoteamento insuficiente.")
        if pivo_linha != k:
            print(f"Trocando linha {k} com linha {pivo_linha} por pivoteamento")
            A[k], A[pivo_linha] = A[pivo_linha], A[k]
            b[k], b[pivo_linha] = b[pivo_linha], b[k]
            imprimir_matriz_b(A, b)

        for i in range(k + 1, n):
            fator = A[i][k] / A[k][k]
            print(f"Multiplicando linha {k} por {fator:.4f} e subtraindo da linha {i}")
            for j in range(k, n):
                A[i][j] -= fator * A[k][j]
            b[i] -= fator * b[k]
            imprimir_matriz_b(A, b)

    # Retro-substituição
    x = [0] * n
    for i in range(n - 1, -1, -1):
        soma = sum(A[i][j] * x[j] for j in range(i + 1, n))
        x[i] = (b[i] - soma) / A[i][i]

    return x, A, b

def calcular_residuo(A, x, b):
    n = len(A)
    residuo = [0] * n
    for i in range(n):
        soma = 0
        for j in range(n):
            soma += A[i][j] * x[j]
        val = soma - b[i]
        residuo[i] = 0 if abs(val) < 1e-5 else val
    return residuo

# Sistema de exemplo
equacoes = [
    '1*x1 - 3*x2 + 2*x3 = 11',
    '-2*x1 + 8*x2 - 3*x3 = -15',
    '4*x1 - 6*x2 + 5*x3 = 29'
]

A, b, vars = montar_sistema(equacoes)
solucao, A_triangular, b_modificado = eliminacao_gauss_pivoteamento(A, b)

print("Solução do sistema:")
for var, val in zip(vars, solucao):
    print(f"{var} = {val:.4f}")

residuo = calcular_residuo(A_triangular, solucao, b_modificado)
print("\nResíduo (Ax - b):")
for i, r in enumerate(residuo):
    print(f"r{i+1} = {r:.4e}" if r != 0 else f"r{i+1} = 0")


Sistema inicial:
Matriz A | Vetor b
[     1.0000    -3.0000     2.0000 ] |    11.0000
[    -2.0000     8.0000    -3.0000 ] |   -15.0000
[     4.0000    -6.0000     5.0000 ] |    29.0000

Trocando linha 0 com linha 2 por pivoteamento
Matriz A | Vetor b
[     4.0000    -6.0000     5.0000 ] |    29.0000
[    -2.0000     8.0000    -3.0000 ] |   -15.0000
[     1.0000    -3.0000     2.0000 ] |    11.0000

Multiplicando linha 0 por -0.5000 e subtraindo da linha 1
Matriz A | Vetor b
[     4.0000    -6.0000     5.0000 ] |    29.0000
[     0.0000     5.0000    -0.5000 ] |    -0.5000
[     1.0000    -3.0000     2.0000 ] |    11.0000

Multiplicando linha 0 por 0.2500 e subtraindo da linha 2
Matriz A | Vetor b
[     4.0000    -6.0000     5.0000 ] |    29.0000
[     0.0000     5.0000    -0.5000 ] |    -0.5000
[     0.0000    -1.5000     0.7500 ] |     3.7500

Multiplicando linha 1 por -0.3000 e subtraindo da linha 2
Matriz A | Vetor b
[     4.0000    -6.0000     5.0000 ] |    29.0000
[     0.0000   